#EasyFinance

**import das bibliotecas utilizadas no código**

In [468]:
#!pip install prettytable


In [469]:
import csv
import json
from math import pow
from datetime import datetime
from prettytable import PrettyTable

##Classes:

As duas classes seguintes implementam a lógica do tratamento de erros:

In [470]:
#exceçoes personalizadas

class EscolhaMenuInvalido(Exception):
    def __init__(self, message = 'Valor inválido! Escolha uma opção entre as disponíveis no menu'):
        self.message = message
        super().__init__(self.message)

class DataInvalida(Exception):
    def __init__(self, message = 'Data inserida foi invalida! Digite uma data possível, utilizando caracteres numericos'):
        self.message = message
        super().__init__(self.message)

A classe principal 'Finanças' onde estão descritos todos os métodos necessários para o funcionamento do CRUD:

1. Os métodos **criar_registros** e **calcular_montante_investido**, respectivamente, criam todos os três tipos de registros (receita, despesa, investimento). Outro método é invocado para calcular o valor do montante.

2. O método **ler_registro** é responsável por exibir todos os registros armazenados. Essa função pode exibir todos os registros, todas as receitas, todas as despesas ou todos os investimentos.

3. O método **exportar_relatorio** é responsável por criar os arquivos CSV e JSON, onde os registros são salvos ao fim da execução do código.

4. O método **atualizar_registro** atualiza um registro conforme é passado o ID do registro.

5. O método **deletar_registro** deleta um registro conforme é passado o ID do registro.

6. O método **carregar_registros_do_arquivo** é responsável por ler todos os registros armazenados no arquivo quando o código é iniciado, se não houver nenhum registro armazenado.

7. O método **recalcular_montante**, quando chamado, faz a atualização do valor do montante em todos os investimentos.

8. O método **agregar_valores** exibe todos os registros compreendidos em um intervalo passado pelo usuário.

9. O método **relatorio_final** exibe várias métricas sobre os registros, como totais, balanço e rendimento.

10.  método **localizar_id** encontra um registro pelo ID passado pelo usuário.

11. O método **agrupar_por** retorna os totais agrupados pela categoria escolhida (tipo de transação, ano ou mês).



In [471]:
class Financas:
    #construtor da classe financas
    def __init__(self):
        self.registros = []
        self.rendimento_investimento = {}
        self.carregar_registros_do_arquivo()

    def criar_registro(self, ano, mes, dia, tipo, valor):
      #convertendo os valores numericos para seus respectivos valores
        valores = {'1': "receita", '2':"despesa", '3':"investimento"}
        tipo = valores[tipo]
        print(tipo)


        id = 1
        if self.registros:
            id = int(self.registros[-1]['id']) + 1

        str(id)

        montante = 0
        if tipo == 'investimento':
            montante = self.calcular_montante_investimento(ano, mes, dia, valor)
            registro = {
              'id' : id,
              'ano': ano,
              'mes': mes,
              'dia': dia,
              'tipo': tipo,
              'valor': valor,
              'montante': montante
        }
        elif tipo == 'despesa':
            valor = valor*(-1)
            registro = {
            'id' : id,
            'ano': ano,
            'mes': mes,
            'dia': dia,
            'tipo': tipo,
            'valor': valor,
        }
        else:

            registro = {
              'id' : id,
              'ano': ano,
              'mes': mes,
              'dia': dia,
              'tipo': tipo,
              'valor': valor,
            }

        self.registros.append(registro)

    def calcular_montante_investimento(self, ano, mes, dia, valor):
            data_investimento = datetime(ano, mes, dia)
            dias_passados = (datetime.now() - data_investimento).days
            montante = valor * pow(1 + 0.000333, dias_passados)
            return montante

    def ler_registros(self, filtro=None):
            if not filtro:
                return self.registros
            return [registro for registro in self.registros if any(value == filtro for value in registro.values())]

    def exportar_relatorio(self, formato='json'):
        if formato == 'json':
            with open('relatorio_financas.json', 'w') as arquivo_json:
                json.dump(self.registros, arquivo_json, indent=2)
        elif formato == 'csv':
            with open('relatorio_financas.csv', 'w', newline='') as arquivo_csv:
                colunas = ['id','ano','mes' ,'dia' , 'tipo', 'valor', 'montante']
                escritor_csv = csv.DictWriter(arquivo_csv, fieldnames=colunas)
                escritor_csv.writeheader()
                escritor_csv.writerows(self.registros)

    def atualizar_registro(self, indice, novo_valor, novo_tipo, novo_ano, novo_mes, novo_dia):
        #convertendo os valores numericos para seus respectivos valores
        valores = {'1': "receita", '2':"despesa", '3':"investimento"}
        novo_tipo = valores[novo_tipo]

        if novo_tipo == 'despesa':
            novo_valor = novo_valor*(-1)

        for i, registro in enumerate(self.registros):
            if int(registro['id']) == int(indice):
                if novo_tipo == 'investimento':
                    registro['montante'] = self.calcular_montante_investimento(novo_ano, novo_mes, novo_dia , novo_valor)
                elif registro['tipo'] == 'investimento' and novo_tipo != 'investimento':
                    registro.pop('montante', None)

                registro['valor'] = novo_valor
                registro['tipo'] = novo_tipo
                registro['dia'] = novo_dia
                registro['mes'] = novo_mes
                registro['ano'] = novo_ano

    def deletar_registro(self, id_a):
        for i, registro in enumerate(self.registros):
            if str(registro.get('id')) == str(id_a):
                del self.registros[i]
                return True

        return False

    def carregar_registros_do_arquivo(self):
        try:
            with open('relatorio_financas.json', 'r') as arquivo_json:
              self.registros = json.load(arquivo_json)
        except FileNotFoundError:
            pass

    def recalcular_montante(self):
            lista = self.ler_registros('investimento')
            i = 0
            while i < len(lista):
                lista[i]['montante'] = self.calcular_montante_investimento(lista[i]['ano'] , lista[i]['mes'] , lista[i]['dia'] , lista[i]['valor'])
                i += 1

            print(self.ler_registros('investimento'))

    def agregar_valores(self,v_inicial,v_final):
            i = 0
            while i < len(self.registros):
                if self.registros[i]['valor'] > v_inicial and self.registros[i]['valor'] < v_final:
                    print(self.registros[i])
                i += 1

    def relatorio_final(self):
        i = 0
        maior = float('-inf')
        menor = float('inf')
        receitas = 0
        despesas = 0
        investimentos = 0
        m_investimentos =0
        while i < len(self.registros):
            #A maior receita
            if self.registros[i]['valor'] > maior:
                maior = self.registros[i]['valor']
                id_maior = self.registros[i]['id']

            #A maior despesa
            if self.registros[i]['valor'] < menor:
                menor = self.registros[i]['valor']
                id_menor = self.registros[i]['id']

            #Total das receitas
            if self.registros[i]['valor'] > 0 and self.registros[i]['tipo'] != 'investimento':
                receitas += self.registros[i]['valor']

            #Total das despesas
            if self.registros[i]['valor'] < 0:
                despesas += self.registros[i]['valor']


            #Total dos investimentos
            if self.registros[i]['valor'] > 0 and self.registros[i]['tipo'] != 'receita':
                investimentos += self.registros[i]['valor']
                m_investimentos += self.registros[i]['montante']

            i += 1

        print(f'''
---------Relatório-----------
Maior receita {maior} de id {id_maior}
Maior despesa {menor} de id {id_menor}
Total investido {investimentos}
Rendimento dos investimentos {m_investimentos - investimentos}
Total das receitas {receitas}
Total das despesas {despesas}
Balanço {receitas + despesas}

              ''')

    def localizar_id(self,id_i):
            existe = False
            for registro in self.registros:
                if str(registro['id']) == id_i:
                    print(registro)
                    return True
            if existe == False:
              raise ValueError



    def despesas(self):
        lista = self.ler_registros('despesa')
        i = 0
        while i < len(lista):
            lista[i]['valor'] *= (-1)
            i += 1

        print(lista)
        print(self.ler_registros('despesa'))

    def agrupar_por(self, categoria):
        totais = {}

        for registro in self.registros:
            valor_categ = registro[categoria]

            if valor_categ not in totais:
                totais[valor_categ] = 0

            totais[valor_categ] += registro['valor']

        return totais


##Funções:

In [472]:
def Menu():
    print('''-------Operações--------
1- Criar registro
2- Ler um registro
3- Atualizar um registro
4- Deletar um registro
5- Atualizar montantes
6- Localizar um registro
7- Agrupar registros
8- Exibir relatório
9- Encerrar''')

def Menu_tipo():
    print('Qual o tipo do registro:\n1-Receita\n2-Despesa\n3-Investimento')

def Menu_data():
    print('Quando ocorreu a operação: aaaa mm dd')

def Menu_valor():
    print('Qual o valor da operação')

def Menu_leitura():
    print('''1- Ler todos os registros
2- Ler somente as receitas
3- Ler somente as despesas
4- Ler somente os investimentos
    ''')


In [473]:
#Esta funcao valida se a data possui valores validos e possiveis de existirem
def validarData(ano, mes, dia):
  if(ano > 2024) or (mes > 12) or dia > 30:
    raise DataInvalida()
  if(ano < 1924) or (mes < 1) or (dia < 1):
    raise DataInvalida()
  if not (isinstance(dia, int) and isinstance(mes, int) and isinstance(ano, int)):
    raise DataInvalida()

###Menu_criar:

In [474]:
def Menu_criar():
    Menu_tipo()
    tipo = input()
    #tratando erro do valor da escolha

    if tipo not in ['1','2','3']:
        raise EscolhaMenuInvalido()
    Menu_data()
    data = input("Digite a data (ano mes dia): ")

    # Dividindo a string nas variáveis ano, mes e dia e convertendo para inteiros
    ano, mes, dia = [int(valor) for valor in data.split()]
    validarData(ano,mes,dia)

    print("Ano:", ano)
    print("Mês:", mes)
    print("Dia:", dia)

    Menu_valor()
    valor = float(input())
    #verificação para não aceitar valores negativos
    if valor < 0:
      raise ValueError
    app.criar_registro(ano,mes,dia,tipo,valor)


###Menu_leitura:

In [475]:
#Habilita a pesquisa por receitas, despejas os investimentos:
def Menu_leitura():
    try:
          leitura = int(input('''1- Ler todos os registros
2- Ler somente as receitas
3- Ler somente as despesas
4- Ler somente os investimentos
    '''))
          if operacoes not in [1,2,3,4]:
              raise EscolhaMenuInvalido()
    except ValueError:
        print('Opção inválida. Por favor, digite um número.')
    except EscolhaMenuInvalido as excecao:
        print(excecao)

    if leitura == 1:
        tabela_b = PrettyTable(['ID', 'Ano', 'Mês', 'Dia', 'Tipo', 'Valor', 'Montante'])

        registros = app.ler_registros()
        for registro in registros:
            montante = registro.get('montante', '')
            tabela_b.add_row([registro['id'], registro['ano'], registro['mes'], registro['dia'], registro['tipo'], registro['valor'], montante])

        print(tabela_b)

    elif leitura == 2:
        tabela_a = PrettyTable(['ID', 'Ano', 'Mês', 'Dia', 'Tipo', 'Valor'])
        registros = app.ler_registros('receita')
        for registro in registros:
            tabela_a.add_row([registro['id'], registro['ano'], registro['mes'], registro['dia'], registro['tipo'], registro['valor']])
        print(tabela_a)

    elif leitura == 3:
        tabela_a = PrettyTable(['ID', 'Ano', 'Mês', 'Dia', 'Tipo', 'Valor'])
        registros = app.ler_registros('despesa')
        for registro in registros:
            tabela_a.add_row([registro['id'], registro['ano'], registro['mes'], registro['dia'], registro['tipo'], registro['valor']])
        print(tabela_a)

    elif leitura == 4:
        tabela_b = PrettyTable(['ID', 'Ano', 'Mês', 'Dia', 'Tipo', 'Valor', 'Montante'])
        registros = app.ler_registros('investimento')
        for registro in registros:
            tabela_b.add_row([registro['id'], registro['ano'], registro['mes'], registro['dia'], registro['tipo'], registro['valor'], registro['montante']])
        print(tabela_b)




###Menu_atualizar:

In [476]:
def Menu_atualizar():
    print('Qual registro deve ser atualizado')
    indice = int(input())
    Menu_tipo()
    tipo = input()
    if tipo not in ['1','2','3']:
      raise EscolhaMenuInvalido()
    Menu_data()
    data = input("Digite a data (ano mes dia): ")
    ano, mes, dia = [int(valor) for valor in data.split()]
    validarData(ano,mes,dia)

    print("Ano:", ano)
    print("Mês:", mes)
    print("Dia:", dia)

    Menu_valor()
    valor = float(input())
    if valor < 0:
      raise ValueError
    app.atualizar_registro(indice ,valor, tipo , ano , mes , dia)


###Menu_deletar:

In [477]:
def Menu_deletar():
    print('Qual registro deve ser deletado:')
    delete = int(input())
    app.deletar_registro(delete)
    tabela = PrettyTable(['ID', 'Ano', 'Mês', 'Dia', 'Tipo', 'Valor', 'Montante'])

    registros = app.ler_registros()
    for registro in registros:
            montante = registro.get('montante', '')
            tabela.add_row([registro['id'], registro['ano'], registro['mes'], registro['dia'], registro['tipo'], registro['valor'], montante])

    print(tabela)


###Menu_localizador:

In [478]:
def Menu_localizador():
    loc_id = input('Digite o id do registro:')
    try:
      app.localizar_id(loc_id)
    except ValueError:
      print("Este id nao existe")

###Menu_agrupar:

In [479]:
def Menu_agrupar():
    print('''\nComo gostaria de agrupar? \n1- Por tipo de investimento
2- Por ano
3- Por mês
    ''')
    try:
      tipo = int(input())
      if tipo not in [1,2,3]:
        raise EscolhaMenuInvalido()
      valores = {1: "tipo", 2:"ano", 3:"mes"}
      tipo = valores[tipo]

      dados = app.agrupar_por(tipo)
      tabela = PrettyTable(['A', 'B'])
      for ano, montante in dados.items():
        tabela.add_row([ano, montante])

      # Exibindo a tabela
      print(tabela)
    except ValueError:
        print('Opção inválida. Por favor, digite um número.')
    except EscolhaMenuInvalido as excecao:
        print(excecao)

##Menu:

In [480]:

app = Financas()
app.carregar_registros_do_arquivo()


while 1:
    Menu()
    operacoes = input()

  #tratando erros do tipo e valor da variavel
    try:
      operacoes = int(operacoes)
      if operacoes not in [1,2,3,4,5,6,7,8,9]:
        raise EscolhaMenuInvalido()
    except ValueError:
        print('Opção inválida. Por favor, digite um número.')
    except EscolhaMenuInvalido as excecao:
      print(excecao)

    try:
      if operacoes == 1:
          Menu_criar()

      elif operacoes == 2:
          Menu_leitura()

      elif operacoes == 3:
          Menu_atualizar()

      elif operacoes == 4:
         Menu_deletar()

      elif operacoes == 5:
          app.recalcular_montante()

      elif operacoes == 6:
          Menu_localizador()

      elif operacoes == 7:
          Menu_agrupar()

      elif operacoes == 8:
          app.relatorio_final()

      elif operacoes == 9:
          app.exportar_relatorio('json')
          app.exportar_relatorio('csv')
          print('Encerrando o código')
          break
      else:
          print('Operação inválida!')
          continue
    except (DataInvalida, EscolhaMenuInvalido) as excecao:
      print(excecao)
    except ValueError as excecao:
        print("Insira um valor válido")


-------Operações--------
1- Criar registro
2- Ler um registro
3- Atualizar um registro
4- Deletar um registro
5- Atualizar montantes
6- Localizar um registro
7- Agrupar registros
8- Exibir relatório
9- Encerrar
4
Qual registro deve ser deletado:
27
+----+------+-----+-----+--------------+-----------+-------------------------+
| ID | Ano  | Mês | Dia |     Tipo     |   Valor   |         Montante        |
+----+------+-----+-----+--------------+-----------+-------------------------+
| 1  | 1976 |  12 |  30 |   despesa    | -88433.84 |                         |
| 2  | 1982 |  5  |  27 |   despesa    | -12274.81 |                         |
| 3  | 1984 |  11 |  24 | investimento |  58626.64 | 2.2299727936818968e+208 |
| 4  | 1985 |  7  |  15 | investimento |  11935.28 | 2.1992729068996347e+204 |
| 5  | 1989 |  3  |  8  | investimento |  15094.99 | 3.1237437153652416e+185 |
| 6  | 1990 |  4  |  3  |   receita    |  14702.01 |                         |
| 7  | 1990 |  6  |  26 |   despesa    |

KeyboardInterrupt: Interrupted by user

In [ ]:
app.agregar_valores(-200000,-8000)